# Uniform Manifold Approximation and Projection (UMAP)

UMAP é um algoritmo de redução de dimensionalidade e visualização de dados que se destaca na preservação tanto da estrutura local quanto da estrutura global dos dados. Baseado em fundamentos da geometria Riemanniana e topologia algébrica, o UMAP constrói uma representação topológica dos dados em alta dimensão e, em seguida, busca otimizar um embedding de baixa dimensão que seja o mais estruturalmente equivalente possível. Este notebook explora os conceitos fundamentais do UMAP e implementa uma versão simplificada do algoritmo para fins didáticos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.neighbors import NearestNeighbors

# Configurações de plotagem
plt.style.use('seaborn-v0_8-whitegrid')

### Geração de Dados Sintéticos

Para visualizar a eficácia do UMAP, utilizaremos um conjunto de dados sintético conhecido como "Swiss Roll". Este dataset é um exemplo clássico de um manifold não linear em três dimensões, onde os pontos seguem uma estrutura espiralada. O objetivo será "desenrolar" este manifold em um espaço bidimensional.

In [ ]:
# Gerar o dataset Swiss Roll
n_samples = 1500
noise = 0.05
X, color = make_swiss_roll(n_samples, noise=noise, random_state=42)

# Normalizar os dados
X = (X - np.min(X)) / (np.max(X) - np.min(X))

### Visualização do Manifold Original

Abaixo, plotamos o dataset em seu espaço tridimensional original. As cores representam a posição intrínseca ao longo do manifold, o que nos ajudará a avaliar se a estrutura foi corretamente preservada após a redução de dimensionalidade.

In [ ]:
# Plotar o Swiss Roll em 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
ax.set_title("Manifold Original do Swiss Roll")
ax.view_init(elev=10., azim=-80)
plt.show()

## Fundamentos Teóricos do UMAP

O UMAP assume que os dados são amostrados de uma variedade Riemanniana (manifold) e que as distâncias entre os pontos na variedade podem ser aproximadas localmente. A partir dessa premissa, o algoritmo opera em duas fases principais: a construção de uma representação topológica dos dados em alta dimensão e a otimização de um layout de baixa dimensão que preserve essa topologia.

### Construção do Grafo de Alta Dimensão

A primeira etapa do UMAP consiste em construir um grafo ponderado que represente a estrutura topológica dos dados originais. Para cada ponto de dados $x_i$, o algoritmo identifica seus $k$ vizinhos mais próximos. A ponderação das arestas deste grafo não é meramente a distância, mas sim uma probabilidade de conexão baseada em uma métrica localmente adaptativa.

Para cada ponto $x_i$, uma métrica local é estabelecida ao encontrar um valor $\sigma_i$ tal que a soma das probabilidades de seus vizinhos some um valor constante (geralmente $\log_2(k)$). A probabilidade condicional $p_{j|i}$ de que $x_j$ esteja conectado a $x_i$ é dada por:

$$p_{j|i} = \exp\left(-\frac{d(x_i, x_j) - \rho_i}{\sigma_i}\right)$$

Onde $\rho_i$ é a distância do ponto $x_i$ ao seu vizinho mais próximo. Esta normalização garante que a conectividade seja definida em termos de uma noção local de distância, tornando o algoritmo robusto a variações na densidade dos dados.

As probabilidades condicionais são então simetrizadas para formar a afinidade final $p_{ij}$ no espaço de alta dimensão, através de uma união fuzzy:

$$p_{ij} = p_{j|i} + p_{i|j} - p_{j|i}p_{i|j}$$

Isso resulta em uma matriz de adjacência ponderada que representa um "conjunto simplicial fuzzy" (fuzzy simplicial set), capturando a estrutura topológica do manifold subjacente aos dados.

### Otimização do Embedding de Baixa Dimensão

A segunda fase do UMAP é encontrar uma representação de baixa dimensão dos dados (seja $y_i$ a representação de $x_i$) que melhor se aproxime da estrutura topológica capturada pelo grafo de alta dimensão. Para isso, uma estrutura similar de afinidades, $q_{ij}$, é definida no espaço de baixa dimensão. A função de afinidade utilizada é tipicamente da família das t-distribuições de Student, o que ajuda a evitar a aglomeração de pontos no centro do embedding e a manter uma separação mais clara entre clusters.

A afinidade $q_{ij}$ entre os pontos $y_i$ e $y_j$ é dada por:

$$q_{ij} = \left(1 + a(y_i - y_j)^{2b}\right)^{-1}$$

Os parâmetros $a$ e $b$ são ajustados para aproximar a forma desejada da distribuição de embedding, geralmente com base nos hiperparâmetros `min_dist` e `spread` do UMAP.

O objetivo da otimização é minimizar a divergência entre as duas distribuições de afinidade, $p_{ij}$ e $q_{ij}$. A função de custo utilizada é a Cross-Entropy (Entropia Cruzada):

$$\text{C(X, Y)} = \sum_{i \neq j} \left[ p_{ij} \log\left(\frac{p_{ij}}{q_{ij}}\right) + (1 - p_{ij}) \log\left(\frac{1 - p_{ij}}{1 - q_{ij}}\right) \right]$$

Esta otimização é realizada através de Stochastic Gradient Descent (SGD), onde arestas do grafo de alta dimensão atuam como forças atrativas e pares de pontos não conectados (amostrados negativamente) atuam como forças repulsivas.

### Implementação Didática do UMAP

A seguir, apresentamos uma classe `UMAP` que implementa as ideias centrais discutidas. Esta implementação é simplificada para fins educacionais e não inclui todas as otimizações de performance (como a busca por vizinhos aproximados ou a amostragem negativa eficiente) encontradas na biblioteca `umap-learn`. O foco é demonstrar o fluxo do algoritmo.

In [ ]:
n = X.shape[0]
knn = NearestNeighbors(n_neighbors=3).fit(X)
dists, inds = knn.kneighbors(X)

rho = dists[:, 1]
sigma = np.zeros(n)

rows = np.repeat(np.arange(n), 3)
cols = inds.flatten()

dist = np.linalg.norm(X[rows] - X[cols], axis=1)
dist = np.maximum(0, dist - rho[rows])
p = np.exp(-dist / (sigma[rows] + 1e-8))

p_matrix = np.zeros((n, n))
p_matrix[rows, cols] = p

p_matrix.shape

In [ ]:
class UMAP:
    def __init__(self, n_neighbors=15, n_components=2, n_epochs=200, lr=0.5, random_state=None):
        self.n_neighbors = n_neighbors
        self.n_components = n_components
        self.n_epochs = n_epochs
        self.lr = lr
        self.random_state = np.random.RandomState(random_state)
        self.a, self.b = 1.929, 0.7915  # parâmetros da função de afinidade do UMAP

    def _find_sigma(self, distances, rho):
        """Busca binária para encontrar sigma."""
        target = np.log2(self.n_neighbors)
        low, high = 1e-3, 10.0
        for _ in range(30):
            mid = (low + high) / 2
            psum = np.sum(np.exp(-(np.maximum(0, distances - rho)) / mid))
            if abs(psum - target) < 1e-5:
                return mid
            if psum > target:
                high = mid
            else:
                low = mid
        return mid

    def _build_neighbor_graph(self, X):
        """Etapa 1: constrói o grafo de vizinhança baseado em distâncias."""
        n = X.shape[0]
        
        # Encontra os k vizinhos mais próximos de cada ponto
        knn = NearestNeighbors(n_neighbors=self.n_neighbors).fit(X)
        dists, inds = knn.kneighbors(X)

        # Calcular rho e sigma
        # rho_i: distância mínima não nula (define o raio local de cada ponto)
        # sigma_i: largura da distribuição local (ajustada por busca binária)
        rho = dists[:, 1]
        sigma = np.zeros(n)
        for i in range(n):
            sigma[i] = self._find_sigma(dists[i, 1:], rho[i])

        # Cria pares (i, j) para todas as conexões entre ponto i e seus vizinhos j
        rows = np.repeat(np.arange(n), self.n_neighbors)
        cols = inds.flatten()

        # Calcula distâncias entre cada ponto e seus vizinhos correspondentes
        dist = np.linalg.norm(X[rows] - X[cols], axis=1)

        # Subtrai o raio local rho_i e zera valores negativos
        dist = np.maximum(0, dist - rho[rows])

        # Converte distâncias em probabilidades (função de afinidade exponencial)
        p = np.exp(-dist / (sigma[rows] + 1e-8))

        # União fuzzy (p_ij + p_ji - p_ij*p_ji)
        p_matrix = np.zeros((n, n))
        p_matrix[rows, cols] = p
        p = p_matrix + p_matrix.T - p_matrix * p_matrix.T

        # Extrai apenas pares conectados e normaliza os pesos finais
        rows, cols = np.where(p > 0)
        p = p[rows, cols]
        p /= (p.max() + 1e-8)
        return rows, cols, p

    def _optimize_embedding(self, rows, cols, weights, Y):
        """Etapa 2: otimiza as posições no espaço reduzido."""
        eps = 1e-8
        for _ in range(self.n_epochs):
            # Atração entre vizinhos reais
            i = self.random_state.randint(0, len(weights), len(Y))
            j = cols[i]
            diff = Y[rows[i]] - Y[j]
            dist2 = np.sum(diff**2, axis=1)
            grad = (2 * self.a * self.b * (dist2 + eps)**(self.b - 1) /
                    (1 + self.a * (dist2 + eps)**self.b))[:, None] * diff
            Y[rows[i]] -= self.lr * grad
            Y[j] += self.lr * grad

            # Repulsão entre pares aleatórios
            neg_i = self.random_state.randint(0, len(Y), len(Y))
            neg_j = self.random_state.randint(0, len(Y), len(Y))
            diff = Y[neg_i] - Y[neg_j]
            dist2 = np.sum(diff**2, axis=1)
            grad = (-self.b / (1 + self.a * (dist2 + eps)**self.b))[:, None] * diff
            Y[neg_i] -= self.lr * grad

        return Y

    def fit_transform(self, X):
        """Executa o UMAP simplificado."""
        rows, cols, weights = self._build_neighbor_graph(X)
        Y = self.random_state.normal(scale=0.01, size=(X.shape[0], self.n_components))
        return self._optimize_embedding(rows, cols, weights, Y)

### Executando a Implementação Simplificada

Agora, vamos instanciar nossa classe `SimpleUMAP` e aplicá-la ao dataset Swiss Roll.

In [ ]:
# Instanciar e rodar o UMAP
simple_umap = UMAP(n_neighbors=30, n_components=2, n_epochs=500, random_state=42)
embedding = simple_umap.fit_transform(X)

### Visualização do Resultado

Finalmente, plotamos o embedding bidimensional resultante. Se o algoritmo funcionou corretamente, devemos ver o "rolo" suíço "desenrolado", com as cores progredindo suavemente de um lado para o outro, indicando que tanto a estrutura local de vizinhança quanto a estrutura global do manifold foram preservadas.

In [ ]:
# Plotar o embedding 2D
plt.figure(figsize=(10, 8))
plt.scatter(embedding[:, 0], embedding[:, 1], c=color, cmap=plt.cm.Spectral, s=50)
plt.title("Embedding 2D do Swiss Roll com SimpleUMAP")
plt.xlabel("Componente UMAP 1")
plt.ylabel("Componente UMAP 2")
plt.show()